In [14]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings("ignore")

def pprint(o):
    print(json.dumps(o, indent=2))

In [15]:
import rasa_nlu
import rasa_core
import spacy

In [3]:
!pip3 install rasa_nlu[spacy]
!python3 -m spacy download en_core_web_md
!python3 -m spacy link en_core_web_md en

  Using cached https://files.pythonhosted.org/packages/a7/62/5abd6379159c75d35cf4c6d05ed33dd1bb6738c45c95694a0af402d142ef/rasa_nlu-0.13.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/9b/9642f6ddc50416b7b25bd893e6d74e34d17fc665325b5a5bef6c787905b1/boto3-1.7.72-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/08/0f/7877fc42fff0b9d70b6442df62d53b3868d3a6ad1b876bdb54335b30ff23/coloredlogs-10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ad/c2/b500ea05d5f9f361a562f089fc91f77ed3b4783e13a08a3daf82069b1224/packaging-17.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/2b/2b05bf1d5a9dd450447c9a5df3e118a465e5d3cb12b73b7220a5064a403f/typing-3.6.4-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/77/de/47e35a97b2b05c2fadbec67d44cfcdcd09b8086951b331d82de90d2912da/jsonschema-2.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/

In [4]:
!pip3 install -r requirements.txt
!pip3 install -e .

Obtaining rasa_core from git+https://github.com/RasaHQ/rasa_core.git@d63f55c44cd919d2031311201caa1c26c94cb365#egg=rasa_core (from -r requirements.txt (line 1))
  Skipping because already up-to-date.
Exception:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/usr/lib/python3/dist-packages/pip/commands/install.py", line 353, in run
    wb.build(autobuilding=True)
  File "/usr/lib/python3/dist-packages/pip/wheel.py", line 749, in build
    self.requirement_set.prepare_files(self.finder)
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "/usr/lib/python3/dist-packages/pip/req/req_set.py", line 620, in _prepare_file
    session=self.session, hashes=hashes)
  File "/usr/lib/python3/dist-packages/pip/download.py", line 821, in unpack_url
    hashes=hashes
  File "/usr/lib/python3/dist-packag

In [32]:
!python3 -m spacy.en.download

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/usr/bin/python3: Error while finding module specification for 'spacy.en.download' (ModuleNotFoundError: No module named 'spacy.en')


### Intent and Entity Training

In [27]:
nlu_md = '''
## intent:greet
- hi
- hello
- hi, my name is [Jack](PERSON)
- hello, my name is [Jack](PERSON)
- hey

## intent:tax_due_dates
- When are property taxes due?
- When do I have to pay my taxes?
- tax due dates?
- where are taxes due?
- when do I pay taxes?
- are taxes due soon?
- find property tax due dates
- property tax due dates
- property taxes due
- how often do I have to pay my taxes?
- how many times do I pay taxes?
- when can I pay my taxes?
- when do I pay my taxes?
- when do I pay property taxes?

## intent:goodbye
- Bye
- goodbye
- peace
- later hater
- thank you, bye
- see you later
- see you soon

## intent:thankyou
- Thank you!
- thanks
- thank you
- I appreciate it
- gracias
'''

%store nlu_md > data/nlu_data.md

Writing 'nlu_md' (str) to file 'data/nlu_data.md'.


### Config

In [28]:
config = '''
language: "en"

pipeline:
- name: 'nlp_spacy'
- name: 'tokenizer_spacy'
- name: 'ner_spacy'
- name: 'intent_featurizer_spacy'
- name: 'intent_classifier_sklearn'
'''

%store config > nlu_config.yml

Writing 'config' (str) to file 'nlu_config.yml'.


### Train Rasa NLU Model

In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

training_data = load_data("data/nlu_data.md")

trainer = Trainer(config.load("nlu_config.yml"))

interpreter = trainer.train(training_data)

model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

### Single Test

In [10]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("data/nlu_data.md", model_directory)

NameError: name 'model_directory' is not defined

### Stories

In [11]:
stories = '''
## tax_due_dates_1
* greet
  - utter_greet
* tax_due_dates
  - action_request_info
* thankyou
  -utter_youre_welcome
* goodbye
  - utter_goodbye

## tax_due_dates_3
* greet
  - utter_greet
* goodbye
  - utter_goodbye

## tax_due_dates_4
* greet
  - utter_greet
* tax_due_dates
  - action_request_info
* goodbye
  - utter_goodbye
'''

%store stories > data/stories.md

Writing 'stories' (str) to file 'data/stories.md'.


### Domain

In [12]:
domain = '''
intents:
  - tax_due_dates
  - greet
  - goodbye
  - thankyou

actions:
  - utter_greet
  - utter_goodbye
  - utter_youre_welcome
  - actions.ActionRequestInfo

entities:
  - PERSON
  
slots:
  trial:
    type: text
  PERSON:
    type: text

templates:
  utter_greet:
    - "Hi!"
    - "Hello, {PERSON}!"
    - "Hi {PERSON}, it's great to hear from you!"
    - "Hey {PERSON}, how can I help you?"

  utter_goodbye:
    - "Bye!"
    - "I hope you had a good experience today!"
    - "See you soon!"
    - "Talk to you later!"
    - "Goodbye."

  utter_youre_welcome:
    - "You're welcome, {PERSON}!"
    - "No problem {PERSON}, I'm happy to help."
    - "You're welcome."
    - "No problem!"
'''

%store domain > domain.yml

Writing 'domain' (str) to file 'domain.yml'.


### Visualizing

In [9]:
from IPython.display import Image
from rasa_core.agent import Agent

agent = Agent("domain.yml")
agent.visualize("data/stories.md", "story_graph.png", max_history=2)
Image(filename="story_graph.png")

INFO:apscheduler.scheduler:Scheduler started
C:\Users\vanguiano\Anaconda3\lib\site-packages\pykwalify\core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 185.38it/s, # trackers=1]


ImportError: ('requires pygraphviz ', 'http://pygraphviz.github.io/')

In [22]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

agent = Agent("domain.yml", policies=[MemoizationPolicy(), KerasPolicy()])

training_data = agent.load_data("data/stories.md")

agent.train(
    training_data,
    validation_split = 0.0,
    epochs = 200)

agent.persist("models/dialogue")

C:\Users\vanguiano\Anaconda3\lib\site-packages\pykwalify\core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 144.71it/s, # trackers=4]
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|████████████████████████████████████████████████| 38/38 [00:02<00:00, 13.44it/s, # actions=23]
INFO:rasa_core.featurizers:Created 23 action examples.
Processed actions: 23it [00:00, 159.31it/s, # examples=23]
INFO:rasa_core.policies.memoization:Memoriz

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_4 (Masking)          (None, 5, 14)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                6016      
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 231       
_________________________________________________________________
activation_4 (Activation)    (None, 7)                 0         
Total params: 6,247
Trainable params: 6,247
Non-trainable params: 0
_________________________________________________________________


INFO:rasa_core.policies.keras_policy:Fitting model with 23 total samples and a validation split of 0.0


Epoch 1/200
23/23 [==============================] - 2s 66ms/step - loss: 1.9344 - acc: 0.0870
Epoch 2/200
23/23 [==============================] - 0s 438us/step - loss: 1.9053 - acc: 0.1304
Epoch 3/200
23/23 [==============================] - 0s 353us/step - loss: 1.9142 - acc: 0.1739
Epoch 4/200
23/23 [==============================] - 0s 349us/step - loss: 1.8865 - acc: 0.3043
Epoch 5/200
23/23 [==============================] - 0s 583us/step - loss: 1.8861 - acc: 0.3913
Epoch 6/200
23/23 [==============================] - 0s 262us/step - loss: 1.8900 - acc: 0.3478
Epoch 7/200
23/23 [==============================] - 0s 475us/step - loss: 1.8544 - acc: 0.5652
Epoch 8/200
23/23 [==============================] - 0s 179us/step - loss: 1.8357 - acc: 0.4348
Epoch 9/200
23/23 [==============================] - 0s 0us/step - loss: 1.8377 - acc: 0.4783
Epoch 10/200
23/23 [==============================] - 0s 694us/step - loss: 1.8399 - acc: 0.5217
Epoch 11/200
23/23 [======================

Epoch 86/200
23/23 [==============================] - 0s 342us/step - loss: 1.2254 - acc: 0.5217
Epoch 87/200
23/23 [==============================] - 0s 398us/step - loss: 1.2581 - acc: 0.5217
Epoch 88/200
23/23 [==============================] - 0s 310us/step - loss: 1.2093 - acc: 0.5217
Epoch 89/200
23/23 [==============================] - 0s 280us/step - loss: 1.2392 - acc: 0.5217
Epoch 90/200
23/23 [==============================] - 0s 468us/step - loss: 1.2258 - acc: 0.5217
Epoch 91/200
23/23 [==============================] - 0s 618us/step - loss: 1.2202 - acc: 0.5217
Epoch 92/200
23/23 [==============================] - 0s 475us/step - loss: 1.2510 - acc: 0.5217
Epoch 93/200
23/23 [==============================] - 0s 389us/step - loss: 1.2005 - acc: 0.5217
Epoch 94/200
23/23 [==============================] - 0s 304us/step - loss: 1.2473 - acc: 0.5217
Epoch 95/200
23/23 [==============================] - 0s 1ms/step - loss: 1.2095 - acc: 0.5217
Epoch 96/200
23/23 [============

Epoch 170/200
23/23 [==============================] - 0s 230us/step - loss: 0.7344 - acc: 0.8696
Epoch 171/200
23/23 [==============================] - 0s 326us/step - loss: 0.7550 - acc: 0.7826
Epoch 172/200
23/23 [==============================] - 0s 568us/step - loss: 0.7648 - acc: 0.8261
Epoch 173/200
23/23 [==============================] - 0s 900us/step - loss: 0.7687 - acc: 0.7391
Epoch 174/200
23/23 [==============================] - 0s 430us/step - loss: 0.7484 - acc: 0.7826
Epoch 175/200
23/23 [==============================] - 0s 174us/step - loss: 0.7981 - acc: 0.7826
Epoch 176/200
23/23 [==============================] - 0s 304us/step - loss: 0.7741 - acc: 0.7826
Epoch 177/200
23/23 [==============================] - 0s 327us/step - loss: 0.7811 - acc: 0.6957
Epoch 178/200
23/23 [==============================] - 0s 523us/step - loss: 0.7114 - acc: 0.8261
Epoch 179/200
23/23 [==============================] - 0s 348us/step - loss: 0.7739 - acc: 0.8261
Epoch 180/200
23/23 

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory models/dialogue exists and contains old model files. All files will be overwritten.
INFO:rasa_core.agent:Persisted model to 'C:\Users\vanguiano\Documents\Projects\conversational-interface\models\dialogue'


### Trying it all out

In [13]:
print("Your bot is ready to talk: Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response['text'])

Your bot is ready to talk: Type your messages here or send 'stop'
hi


NameError: name 'agent' is not defined